# Characterizing coherent errors $\epsilon$ via iterative gates

We can make over/under-rotation more pronounced in quantum experiments by applying a sequence of itera

In [5]:
import warnings
warnings.filterwarnings('ignore')
from qiskit.tools.jupyter import *

from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_manila')

from qiskit import pulse, schedule                 
from qiskit.circuit import Parameter, QuantumCircuit, Gate
import numpy as np

In [6]:
backend_config = backend.configuration()
dt = backend_config.dt
backend_defaults = backend.defaults()
backend_properties = backend.properties()
GHz = 1e9

qubit = 0

In [7]:
"""
    The following parameteres were last 
    updated on 13rd September, 2022
    + Qubit: 0
    + Backend: ibmq_manila
    + Pulse instruction: Pi pulse on subspace (0-1)
"""

# Pulse parameters
drive_duration_01 = 544
drive_sigma_01 = 67
drive_amplitude_01 = 0.09265405516803696

### General pulse instruction ###

with pulse.build(backend=backend) as inst_x_halfpi_01:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(backend_defaults.qubit_freq_est[qubit], drive_chan)
    pulse.play(pulse.Gaussian(duration=drive_duration_01,
                              amp=drive_amplitude_01/2,
                              sigma=drive_sigma_01), drive_chan)
    
with pulse.build(backend=backend) as inst_x_fullpi_01:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(backend_defaults.qubit_freq_est[qubit], drive_chan)
    pulse.play(pulse.Gaussian(duration=drive_duration_01,
                              amp=drive_amplitude_01,
                              sigma=drive_sigma_01), drive_chan)

In [8]:
"""
    The following parameteres were last 
    updated on 13rd September, 2022
    + Qubit: 0
    + Backend: ibmq_manila
    + Pulse instruction: Pi pulse on subspace (1-2)
"""

# Pulse parameters
drive_duration_12 = 160
drive_sigma_12 = 40
drive_amplitude_12 = 0.17102045531902912
anhar = backend_properties.qubits[qubit][3].value*GHz

### General pulse instruction ###

with pulse.build(backend=backend) as inst_x_fullpi_12:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(backend_defaults.qubit_freq_est[qubit]+anhar, drive_chan)
    pulse.play(pulse.Gaussian(duration=drive_duration_12,
                              amp=drive_amplitude_12,
                              sigma=drive_sigma_12), drive_chan)

In [9]:
gate_x_halfpi_01 = Gate('$X_{\pi/2}^{(01)}$', 1, [])
gate_x_fullpi_01 = Gate('$X_{\pi}^{(01)}$', 1, [])
gate_x_fullpi_12 = Gate('$X_{\pi}^{(12)}$', 1, [])

In [51]:
experiment_package = []

In [52]:
# 0 state
ground_circs = QuantumCircuit(1, 1)
ground_circs.measure(0, 0)
experiment_package.append(ground_circs)

# 1 state
one_circs = QuantumCircuit(1, 1)
one_circs.append(gate_x_fullpi_01, [0])
one_circs.measure(0, 0)
one_circs.add_calibration(gate_x_fullpi_01, (0,), inst_x_fullpi_01, [])
experiment_package.append(one_circs)

# 2 state
two_circs = QuantumCircuit(1, 1)
two_circs.append(gate_x_fullpi_01, [0])
two_circs.append(gate_x_fullpi_12, [0])
two_circs.measure(0, 0)
two_circs.add_calibration(gate_x_fullpi_01, (0,), inst_x_fullpi_01, [])
two_circs.add_calibration(gate_x_fullpi_12, (0,), inst_x_fullpi_12, [])
experiment_package.append(two_circs)

In [53]:
integer = 48
repetitions = 2*integer+1

for d in range(repetitions):
    qc_epsilon = QuantumCircuit(1,1)
    for j in range(98+d):
        qc_epsilon.append(gate_x_halfpi_01, [0])
    qc_epsilon.measure(0,0)
    qc_epsilon.add_calibration(gate_x_halfpi_01, (0,), inst_x_halfpi_01, [])
    experiment_package.append(qc_epsilon)

In [56]:
from qiskit.tools import job_monitor

epsilon = backend.run(experiment_package,
                  meas_level=1,
                  meas_return='single',
                  shots=20000)
print(epsilon.job_id())
job_monitor(epsilon)

6328174722406153e34d6fa3
Job Status: job has successfully run
